In [1]:
import chseg
import numpy as np
import tensorflow as tf

from sklearn.metrics import classification_report

In [2]:
params = {
    'seq_len': 50,
    'text_iter_step': 500,
    'batch_size': 128,
    'n_class': 4,
    'hidden_dim': 128,
}

In [3]:
def to_test_seq(*args):
    return [x[:(len(x)-len(x)%params['seq_len'])].reshape([-1,params['seq_len']]) for x in args]

def next_train_batch(x_train, y_train):
    window = params['batch_size'] * params['seq_len']
    for i in range(0, len(x_train)-window, params['text_iter_step']):
        yield (x_train[i: i+window].reshape(params['batch_size'], params['seq_len']),
               y_train[i: i+window].reshape(params['batch_size'], params['seq_len']),)
        
def train_input_fn(x_train, y_train):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_train_batch(x_train, y_train),
        (tf.int32, tf.int32),
        (tf.TensorShape([None,params['seq_len']]), tf.TensorShape([None,params['seq_len']])))
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [4]:
x_train, y_train, x_test, y_test, params['vocab_size'], word2idx, idx2word = chseg.load_data()
x_train, y_train, x_test, y_test = np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)
X_test, Y_test = to_test_seq(x_test, y_test)

In [5]:
def forward(x, reuse, is_training):
    with tf.variable_scope('model', reuse=reuse):
        x = tf.contrib.layers.embed_sequence(x, params['vocab_size'], params['hidden_dim'])
        x = tf.layers.dropout(x, 0.1, training=is_training)
        
        pad = tf.zeros([tf.shape(x)[0], 1, params['hidden_dim']])
        for k_sz in [3, 5, 7, 9, 11]:
            n = (k_sz - 1) // 2
            _x = tf.concat([pad]*n + [x] + [pad]*n, 1)
            x += tf.layers.conv1d(_x, params['hidden_dim'], k_sz, activation=tf.nn.relu)

        logits = tf.layers.dense(x, params['n_class'])
    return logits


def model_fn(features, labels, mode, params):
    logits_tr = forward(features, reuse=False, is_training=True)
    logits_te = forward(features, reuse=True, is_training=False)
    seq_lens = tf.count_nonzero(features, 1)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        log_likelihood, _ = tf.contrib.crf.crf_log_likelihood(
            logits_tr, labels, seq_lens)
        
        loss_op = tf.reduce_mean(-log_likelihood)
        
        train_op = tf.train.AdamOptimizer().minimize(
            loss_op, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss_op, train_op=train_op)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        trans_params = tf.get_variable('transitions', [params['n_class'], params['n_class']])
        
        viterbi_seq, _ = tf.contrib.crf.crf_decode(
            logits_te, trans_params, seq_lens)
        
        return tf.estimator.EstimatorSpec(mode, predictions=viterbi_seq)

In [6]:
estimator = tf.estimator.Estimator(model_fn, params=params)

estimator.train(lambda: train_input_fn(x_train, y_train))

Y_pred = np.concatenate(list(estimator.predict(
    tf.estimator.inputs.numpy_input_fn(
        x = X_test,
        batch_size = params['batch_size'],
        shuffle = False))), -1)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp8o7rho2l', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x114b35390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


/usr/local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp8o7rho2l/model.ckpt.
INFO:tensorflow:loss = 71.236336, step = 1
INFO:tensorflow:global_step/sec: 3.91973
INFO:tensorflow:loss = 15.272905, step = 101 (25.513 sec)
INFO:tensorflow:global_step/sec: 3.87775
INFO:tensorflow:loss = 13.346983, step = 201 (25.788 sec)
INFO:tensorflow:global_step/sec: 3.57412
INFO:tensorflow:loss = 9.473921, step = 301 (27.979 sec)
INFO:tensorflow:global_step/sec: 3.82842
INFO:tensorflow:loss = 9.3798485, step = 401 (26.120 sec)
INFO:tensorflow:global_step/sec: 3.95771
INFO:tensorflow:loss = 11.199088, step = 501 (25.267 sec)
INFO:tensorflow:global_step/sec: 3.85074
INFO:tensorflow:loss = 10.206051, step = 601 (25.969 sec)
INFO:tensorflow:global_step/sec: 3.88912
I

In [7]:
print(classification_report(Y_test.ravel(), Y_pred.ravel(), target_names=['B','M','E','S']))

sample = '我来到大学读书，希望学到知识'
labels = list(estimator.predict(
    tf.estimator.inputs.numpy_input_fn(
        x = np.atleast_2d([word2idx[w] for w in sample] + [0]*(params['seq_len']-len(sample))),
        shuffle = False)))[0]

labels = labels[:len(sample)]
res = ''
for i, l in enumerate(labels):
    c = sample[i]
    if l == 2 or l == 3:
        c += ' '
    res += c
print(res)

             precision    recall  f1-score   support

          B       0.92      0.95      0.93    116058
          M       0.84      0.76      0.80     25425
          E       0.93      0.95      0.94    116057
          S       0.93      0.90      0.92    106810

avg / total       0.92      0.92      0.92    364350

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp8o7rho2l/model.ckpt-2912
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
我 来到 大学 读书 ， 希望 学 到 知识
